In [ ]:
# 0-) Ülkeler

# Ülkeleri seçili yılda üyelere göre filtrelememiz için kod
import pandas as pd

def add_membership_column(df):
    rows = []
    for index, row in df.iterrows():
        nato_year = row['NATO'] if row['NATO'] != '-' else None
        eu_year = row['EU'] if row['EU'] != '-' else None
        
        # 2000-2024 arasında NATO ya da EU üyesi olup olmadığını gösterecek kod
        for year in range(2000, 2025):
            nato_membership = 'Yes' if nato_year and int(nato_year) <= year else 'No'
            eu_membership = 'Yes' if eu_year and int(eu_year) <= year else 'No'
            new_row = row.to_dict()
            new_row['Year'] = year
            new_row['NATO_Membership'] = nato_membership
            new_row['EU_Membership'] = eu_membership
            rows.append(new_row)
    
    # Dataframe oluşturuyoruz
    expanded_df = pd.DataFrame(rows)
    
    # Yılları başa çekiyoruz panel veri formatı için
    cols = ['Year'] + [col for col in expanded_df.columns if col != 'Year']
    expanded_df = expanded_df[cols]
    return expanded_df

# Ülkelere uyguluyoruz
file_path = '0-) Ulkeler.xlsx'
excel_data = pd.ExcelFile(file_path)
df = excel_data.parse('Sheet1')
expanded_df = add_membership_column(df)
expanded_df = expanded_df.drop(['NATO', 'EU'], axis=1)

In [ ]:
# 1-) Yeni Ürünlerin Sektörlerini Bulmak

import pandas as pd

def load_and_process_new_products():
    # Kullanacağımız yılları tanımlıyoruz
    years = [2002, 2007, 2012, 2017, 2022]
    file_paths = {
        2002: r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS02_V202401\product_codes_HS02_V202401.csv",
        2007: r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS07_V202401\product_codes_HS07_V202401.csv",
        2012: r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS12_V202401\product_codes_HS12_V202401.csv",
        2017: r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS17_V202401\product_codes_HS17_V202401.csv",
        2022: r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS22_V202401\product_codes_HS22_V202401.csv"
    }
    
    # Her yıl için ürün kodlarını ekliyoruz
    dfs = {year: pd.read_csv(file_paths[year]) for year in years}
    product_sets = {year: set(dfs[year]['code']) for year in years}

    # Yeni ürünleri buluyoruz
    new_products_dfs = []
    for i in range(1, len(years)):
        year = years[i]
        prev_year = years[i - 1]
        new_products = product_sets[year] - product_sets[prev_year]
        
        # Yıl kolonunu ekliyoruz
        new_df = dfs[year][dfs[year]['code'].isin(new_products)].copy()
        new_df['Yıl'] = year
        new_products_dfs.append(new_df)
    
    # Bütün yıllardaki yeni ürünleri tek dataframede topluyoruz
    combined_new_products = pd.concat(new_products_dfs, ignore_index=True)
    
    # 2 basamaklı HS kodunu hesaplıyoruz
    combined_new_products['2 Basamaklı HS Kodu'] = combined_new_products['code'] // 10000
    
    # Sektör kodlarını ve isimilerini kullanarak birleştiriyoruz
    atlas_df = pd.read_excel('0-) HS92 Sektor Kodlari ve Isimleri.xlsx')
    atlas_df['hs2'] = atlas_df['hs2'].astype(str).str.zfill(2)
    combined_new_products['2 Basamaklı HS Kodu'] = combined_new_products['2 Basamaklı HS Kodu'].astype(str).str.zfill(2)
    combined_new_products = combined_new_products.merge(
        atlas_df[['hs2', 'hs2_name', 'sector']],
        left_on='2 Basamaklı HS Kodu',
        right_on='hs2',
        how='left'
    ).rename(columns={
        'code': '6 Basamaklı HS Kodu',
        'hs2_name': '2 Basamaklı HS İsmi',
        'sector': 'Sektör İsmi'
    }).drop(columns=['hs2'])

    # Yıl sütununu en başa alıyoruz
    cols = ['Yıl'] + [col for col in combined_new_products.columns if col != 'Yıl']
    combined_new_products = combined_new_products[cols]
    return combined_new_products

processed_new_products = load_and_process_new_products()

# Kaydediyoruz
processed_new_products.to_excel("1-) Yeni Urunlerin Sektorleri.xlsx", index=False)

KeyError: 'sector'

In [ ]:
# Her seferinde vakit kaybetmeyeyim diye dış ticaret verilerini bir data frame'e kaydediyorum

import pandas as pd

# Yıllar ve HS versiyonları
yillar = list(range(2012, 2022))
hs_versiyonlari = {year: 'HS12' if year <= 2016 else 'HS17' for year in yillar}

# Dosya yolu formatı
dosya_yolu_format = r"C:\Users\oguzo\Desktop\MENA\BACI\BACI_{hs_version}_V202401\BACI_{hs_version}_Y{year}_V202401.csv"
# Tüm yılları içeren DataFrame'i birleştir
combined_atlas_df = pd.read_excel('0-) HS92 Sektor Kodlari ve Isimleri.xlsx')

# Ülkeleri yükle
ulkeler_df = pd.read_excel('0-) Ulkeler.xlsx')  
combined_atlas_df['sector'] = [zz.strip() for zz in combined_atlas_df['sector']] 
combined_atlas_df['hs2_name'] = [zz.strip() for zz in combined_atlas_df['hs2_name']] 

# Her yılın verisini oku ve birleştir
for year in [yillar[0]]:
    hs_version = hs_versiyonlari[year] 
    dosya_yolu = dosya_yolu_format.format(hs_version=hs_version, year=year)  
    df = pd.read_csv(dosya_yolu)
    df['hs2'] = df['k']//10000
    df = df.merge(combined_atlas_df)

    ### RCA HS6
    x_p = df[['t','i', 'k','v']].groupby(['t','i','k']).sum().reset_index()
    x_p.columns = ['t','i','k','prod_country']

    x_c = df[['t','i', 'v']].groupby(['t','i']).sum().reset_index()
    x_c.columns = ['t','i','country']

    w_p = df[['t', 'k','v']].groupby(['t','k']).sum().reset_index()
    w_p.columns = ['t','k','prod']

    w = df['v'].sum()

    rca = x_p.merge(x_c).merge(w_p)
    rca['w'] = w
    rca['RCA_HS6'] = (rca['prod_country']/rca['country'])/(rca['prod']/rca['w'])

    df = df.merge(rca[['t','i','k','RCA_HS6']])

    ### RCA HS2

    x_p = df[['t','i', 'hs2','v']].groupby(['t','i','hs2']).sum().reset_index()
    x_p.columns = ['t','i','hs2','prod_country']

    x_c = df[['t','i', 'v']].groupby(['t','i']).sum().reset_index()
    x_c.columns = ['t','i','country']

    w_p = df[['t', 'hs2','v']].groupby(['t','hs2']).sum().reset_index()
    w_p.columns = ['t','hs2','prod']

    w = df['v'].sum()

    rca = x_p.merge(x_c).merge(w_p)
    rca['w'] = w
    rca['RCA_HS2'] = (rca['prod_country']/rca['country'])/(rca['prod']/rca['w'])

    df = df.merge(rca[['t','i','hs2','RCA_HS2']])


    ### RCA Sector

    x_p = df[['t','i', 'sector','v']].groupby(['t','i','sector']).sum().reset_index()
    x_p.columns = ['t','i','sector','prod_country']

    x_c = df[['t','i', 'v']].groupby(['t','i']).sum().reset_index()
    x_c.columns = ['t','i','country']

    w_p = df[['t', 'sector','v']].groupby(['t','sector']).sum().reset_index()
    w_p.columns = ['t','sector','prod']

    w = df['v'].sum()

    rca = x_p.merge(x_c).merge(w_p)
    rca['w'] = w
    rca['RCA_sector'] = (rca['prod_country']/rca['country'])/(rca['prod']/rca['w'])

    df = df.merge(rca[['t','i','sector','RCA_sector']])

    df = df[df['i'].isin(ulkeler_df['country_code'])]

In [6]:
df

,t,i,j,k,v,q,hs2,sector,hs2_name,RCA_HS6,RCA_HS2,RCA_sector
2460,2012,8,12,121190,1.107,0.012,12,Agriculture,Oil seeds and oleaginous fruits,84.805741,2.463455,0.978607
2461,2012,8,12,240110,0.519,0.030,24,Agriculture,Tobacco,4.741012,1.261261,0.978607
2462,2012,8,12,611090,1.199,0.042,61,Textiles,"Apparel, knit",0.635283,6.635385,5.343747
2463,2012,8,12,620349,0.383,0.013,62,Textiles,"Apparel, not knit",7.483516,7.857122,5.343747
2464,2012,8,12,620590,9.121,0.070,62,Textiles,"Apparel, not knit",11.239501,7.857122,5.343747
...,...,...,...,...,...,...,...,...,...,...,...,...
9034690,2012,887,842,970600,630.891,NA,97,Textiles,Art,0.400568,0.059478,0.010338
9034691,2012,887,860,630900,0.100,0.045,63,Textiles,Other made up textile articles,0.006669,0.003845,0.010338
9034692,2012,887,894,190531,10.648,19.440,19,Agriculture,"Preparations of cereals, flour, starch or milk",4.319722,0.591304,0.621365
9034693,2012,887,894,300490,2.005,0.044,30,Chemicals,Pharmaceutical products,0.023380,0.025288,0.065408


In [ ]:
# Her seferinde vakit kaybetmeyeyim diye dış ticaret verilerini bir data frame'e kaydediyorum

import pandas as pd

# Yıllar ve HS versiyonları
yillar = list(range(2012, 2022))
hs_versiyonlari = {year: 'HS12' if year <= 2016 else 'HS17' for year in yillar}

# Dosya yolu formatı
dosya_yolu_format = r"C:\Users\oguzo\Desktop\BACI\BACI_{hs_version}_V202401\BACI_{hs_version}_Y{year}_V202401.csv"

# Tüm yılların verilerini tutacak liste
tum_yillar_verisi = []

# Her yılın verisini oku ve birleştir
for year in yillar:
    hs_version = hs_versiyonlari[year] 
    dosya_yolu = dosya_yolu_format.format(hs_version=hs_version, year=year)  
    df = pd.read_csv(dosya_yolu)
    tum_yillar_verisi.append(df)

# Sonuç DataFrame'i
birlesik_df

In [ ]:
# Her seferinde vakit kaybetmeyeyim diye dış ticaret verilerini bir data frame'e kaydediyorum

import pandas as pd

# Yıllar ve HS versiyonları
yillar = list(range(2012, 2022))
hs_versiyonlari = {year: 'HS12' if year <= 2016 else 'HS17' for year in yillar}

# Dosya yolu formatı
dosya_yolu_format = r"C:\Users\oguzo\Desktop\MENA\BACI\BACI_{hs_version}_V202401\BACI_{hs_version}_Y{year}_V202401.csv"
# Tüm yılları içeren DataFrame'i birleştir

combined_atlas_df = pd.read_excel('0-) HS92 Sektör Kodları ve İsimleri.xlsx')


# Ülkeleri yükle
ulkeler_df = pd.read_excel('0-) Ülkeler.xlsx')  

combined_atlas_df['sector'] = [zz.strip() for zz in combined_atlas_df['sector']] 

combined_atlas_df['hs2_name'] = [zz.strip() for zz in combined_atlas_df['hs2_name']] 

# Her yılın verisini oku ve birleştir
for year in [yillar[0]]:
    hs_version = hs_versiyonlari[year] 
    dosya_yolu = dosya_yolu_format.format(hs_version=hs_version, year=year)  
    df = pd.read_csv(dosya_yolu)

    df['hs2'] = df['k']//10000

    df = df.merge(combined_atlas_df)


    ### RCA HS6

    x_p = df[['t','i', 'k','v']].groupby(['t','i','k']).sum().reset_index()
    x_p.columns = ['t','i','k','prod_country']

    x_c = df[['t','i', 'v']].groupby(['t','i']).sum().reset_index()
    x_c.columns = ['t','i','country']

    w_p = df[['t', 'k','v']].groupby(['t','k']).sum().reset_index()
    w_p.columns = ['t','k','prod']

    w = df['v'].sum()

    rca = x_p.merge(x_c).merge(w_p)
    rca['w'] = w
    rca['RCA_HS6'] = (rca['prod_country']/rca['country'])/(rca['prod']/rca['w'])

    df = df.merge(rca[['t','i','k','RCA_HS6']])

    ### RCA HS2

    x_p = df[['t','i', 'hs2','v']].groupby(['t','i','hs2']).sum().reset_index()
    x_p.columns = ['t','i','hs2','prod_country']

    x_c = df[['t','i', 'v']].groupby(['t','i']).sum().reset_index()
    x_c.columns = ['t','i','country']

    w_p = df[['t', 'hs2','v']].groupby(['t','hs2']).sum().reset_index()
    w_p.columns = ['t','hs2','prod']

    w = df['v'].sum()

    rca = x_p.merge(x_c).merge(w_p)
    rca['w'] = w
    rca['RCA_HS2'] = (rca['prod_country']/rca['country'])/(rca['prod']/rca['w'])

    df = df.merge(rca[['t','i','hs2','RCA_HS2']])


    ### RCA Sector

    x_p = df[['t','i', 'sector','v']].groupby(['t','i','sector']).sum().reset_index()
    x_p.columns = ['t','i','sector','prod_country']

    x_c = df[['t','i', 'v']].groupby(['t','i']).sum().reset_index()
    x_c.columns = ['t','i','country']

    w_p = df[['t', 'sector','v']].groupby(['t','sector']).sum().reset_index()
    w_p.columns = ['t','sector','prod']

    w = df['v'].sum()

    rca = x_p.merge(x_c).merge(w_p)
    rca['w'] = w
    rca['RCA_sector'] = (rca['prod_country']/rca['country'])/(rca['prod']/rca['w'])

    df = df.merge(rca[['t','i','sector','RCA_sector']])

    df = df[df['i'].isin(ulkeler_df['country_code'])]

In [37]:
df

,t,i,j,k,v,q,hs2,sector,hs2_name,RCA_HS6,RCA_HS2,RCA_sector
0,2012,4,8,610469,2.734,0.044,61,Textiles,"Apparel, knit",0.135622,0.023182,2.451609
1,2012,4,8,630900,0.110,0.082,63,Textiles,Other made up textile articles,0.329029,0.091198,2.451609
2,2012,4,12,91099,1.185,1.200,9,Agriculture,"Coffee, tea and spices",1.872373,3.641572,1.982568
3,2012,4,32,392630,1.288,0.123,39,Chemicals,Plastics,0.093851,0.012163,0.019989
4,2012,4,32,710310,0.096,0.004,71,Stone,Precious metals and stones,80.408926,8.353549,6.883085
...,...,...,...,...,...,...,...,...,...,...,...,...
9041664,2012,894,854,843041,751.268,18.000,84,Machinery,Industrial Machinery,0.660853,0.069335,0.053925
9041665,2012,894,854,846799,28.759,0.343,84,Machinery,Industrial Machinery,0.019369,0.069335,0.053925
9041666,2012,894,854,848140,9.950,1.341,84,Machinery,Industrial Machinery,0.014115,0.069335,0.053925
9041667,2012,894,858,240110,77.579,17.325,24,Agriculture,Tobacco,53.613649,5.966768,0.898969


In [7]:
# 2-) 1 Basamaklı sektörlerin ticaret hacimleri

import pandas as pd

# İlk olarak, her yıl ve k_1_name (sektör adı) bazında toplam değerleri bul
yearly_totals = birlesik_df.groupby(['t', 'k_1_name'])['v'].sum().reset_index(name='sector_total')

# Yıllık toplamı, her yıl için genel toplamı hesaplamak için kullan
yearly_totals['year_total'] = yearly_totals.groupby('t')['sector_total'].transform('sum')

# Her sektörün o yılki toplam içerisindeki oranını hesapla
yearly_totals['k_1_oran'] = yearly_totals['sector_total'] / yearly_totals['year_total']
final_df = yearly_totals[['t', 'k_1_name', 'k_1_oran']]
final_df.columns = ['Yıl', '1 Basamaklı HS İsimleri', 'Ticaret Hacimleri']

# Excel dosyasına kaydet
final_df.to_excel("2-) 1 Basamaklı Sektörlerin Ticaret Hacimleri.xlsx", index=False)

In [8]:
# 3-) 2 Basamaklı sektörlerin ticaret hacimleri

import pandas as pd

# İlk olarak, her yıl ve k_2_name (sektör adı) bazında toplam değerleri bul
yearly_totals = birlesik_df.groupby(['t', 'k_2_name'])['v'].sum().reset_index(name='sector_total')

# Yıllık toplamı, her yıl için genel toplamı hesaplamak için kullan
yearly_totals['year_total'] = yearly_totals.groupby('t')['sector_total'].transform('sum')

# Her sektörün o yılki toplam içerisindeki oranını hesapla
yearly_totals['k_2_oran'] = yearly_totals['sector_total'] / yearly_totals['year_total']
final_df = yearly_totals[['t', 'k_2_name', 'k_2_oran']]
final_df.columns = ['Yıl', '2 Basamaklı HS İsimleri', 'Ticaret Hacimleri']

# Excel dosyasına kaydet
final_df.to_excel("3-) 2 Basamaklı Sektörlerin Ticaret Hacimleri.xlsx", index=False)

In [3]:
# 4-) 1 Basamaklı sektörlerin ülkelerdeki ticaret hacimleri

import pandas as pd

# Ülkeleri yükle
ulkeler_df = pd.read_excel('0-) Ülkeler.xlsx')  

# Ülkeleri filtrele kendi aralarındaki ithalat ve ihracat için
filtered_countries = ulkeler_df['country_code'].unique() 

# Yıl bazında veriyi parçalara bölerek işlem yap
years = birlesik_df['t'].unique()  
final_dfs = []

for year in years:
    # Veriyi yıl bazında filtrele
    df_year = birlesik_df[birlesik_df['t'] == year]

    # Hem ihracatçı hem de ithalatçı ülkeleri filtrele
    df_filtered_year = df_year[(df_year['i'].isin(filtered_countries)) & (df_year['j'].isin(filtered_countries))]
    
    # Yıl ve ülke bazında sektörlerin ticaret hacimlerini hesapla
    yearly_totals = df_filtered_year.groupby(['t', 'i', 'k_1_name'])['v'].sum().reset_index(name='sector_total')
    
    # Her yıl için toplam ticaret hacmini hesapla
    yearly_totals['year_total'] = yearly_totals.groupby(['i'])['sector_total'].transform('sum')
    
    # Her sektörün o yılki ticaret hacmindeki oranını hesapla
    yearly_totals['k_1_oran'] = yearly_totals['sector_total'] / yearly_totals['year_total']
    
    # Sonuçları listeye ekle
    final_dfs.append(yearly_totals)

# Parçaları birleştir
final_df = pd.concat(final_dfs, ignore_index=True)

# Ülke kodlarını ülke adlarına dönüştürme
final_df = final_df.merge(ulkeler_df[['country_code', 'Countries']], left_on='i', right_on='country_code', how='left')

# Ülke kodu yerine adı yaz
final_df['Ülke'] = final_df['Countries']

# Gereksiz sütunları çıkar
final_df = final_df[['t', 'Ülke', 'k_1_name', 'sector_total', 'year_total', 'k_1_oran']]

# Sütun adlarını düzenle
final_df.columns = ['Yıl', 'Ülke', '1 Basamaklı HS İsimleri', 'Sektör Ticaret Hacmi', 'Genel Ticaret Hacmi', 'Ticaret Hacmi Oranı']

# Excele kaydet
final_df.to_excel("4-) Ülke Bazlı 1 Basamaklı HS Sektörlerinin Oranları.xlsx", index=False)

,Yıl,Ülke,1 Basamaklı HS İsimleri,Sektör Ticaret Hacmi,Genel Ticaret Hacmi,Ticaret Hacmi Oranı
0,2012,Albania,Agriculture (1 HS92),49041.844,528202.686,0.092847
1,2012,Albania,Chemicals (5 HS92),11598.101,528202.686,0.021958
2,2012,Albania,Electronics (8 HS92),6873.537,528202.686,0.013013
3,2012,Albania,Machinery (7 HS92),14995.406,528202.686,0.028389
4,2012,Albania,Metals (4 HS92),190887.841,528202.686,0.361391
...,...,...,...,...,...,...
4495,2021,Yemen,Metals (4 HS92),64316.707,656100.045,0.098029
4496,2021,Yemen,Minerals (3 HS92),422.771,656100.045,0.000644
4497,2021,Yemen,Stone (2 HS92),80920.642,656100.045,0.123336
4498,2021,Yemen,Textiles (0 HS92),448.823,656100.045,0.000684


In [6]:
# 5-) 2 Basamaklı sektörlerin ülkelerdeki ticaret hacimleri

import pandas as pd

# Ülkeleri yükle
ulkeler_df = pd.read_excel('0-) Ülkeler.xlsx')  

# Ülkeleri filtrele kendi aralarındaki ithalat ve ihracat için
filtered_countries = ulkeler_df['country_code'].unique() 

# Yıl bazında veriyi parçalara bölerek işlem yap
years = birlesik_df['t'].unique()  
final_dfs = []

for year in years:
    # Veriyi yıl bazında filtrele
    df_year = birlesik_df[birlesik_df['t'] == year]

    # Hem ihracatçı hem de ithalatçı ülkeleri filtrele
    df_filtered_year = df_year[(df_year['i'].isin(filtered_countries)) & (df_year['j'].isin(filtered_countries))]
    
    # Yıl ve ülke bazında sektörlerin ticaret hacimlerini hesapla
    yearly_totals = df_filtered_year.groupby(['t', 'i', 'k_2_name'])['v'].sum().reset_index(name='sector_total')
    
    # Her yıl için toplam ticaret hacmini hesapla
    yearly_totals['year_total'] = yearly_totals.groupby(['i'])['sector_total'].transform('sum')
    
    # Her sektörün o yılki ticaret hacmindeki oranını hesapla
    yearly_totals['k_2_oran'] = yearly_totals['sector_total'] / yearly_totals['year_total']
    
    # Sonuçları listeye ekle
    final_dfs.append(yearly_totals)

# Parçaları birleştir
final_df = pd.concat(final_dfs, ignore_index=True)

# Ülke kodlarını ülke adlarına dönüştürme
final_df = final_df.merge(ulkeler_df[['country_code', 'Countries']], left_on='i', right_on='country_code', how='left')

# Ülke kodu yerine adı yaz
final_df['Ülke'] = final_df['Countries']

# Gereksiz sütunları çıkar
final_df = final_df[['t', 'Ülke', 'k_2_name', 'sector_total', 'year_total', 'k_2_oran']]

# Sütun adlarını düzenle
final_df.columns = ['Yıl', 'Ülke', '2 Basamaklı HS İsimleri', 'Sektör Ticaret Hacmi', 'Genel Ticaret Hacmi', 'Ticaret Hacmi Oranı']

# Excele kaydet
final_df.to_excel("5-) Ülke Bazlı 2 Basamaklı HS Sektörlerinin Oranları.xlsx", index=False)

In [ ]:
# 6-) Yeni ür

In [10]:
#denemeve sil
import pandas as pd
import numpy as np

# Numeric veri türlerini daha az bellek kullanan türlere dönüştürelim
for col in birlesik_df.select_dtypes(include=['int', 'float']).columns:
    birlesik_df[col] = pd.to_numeric(birlesik_df[col], downcast='unsigned')

# Yineleyen string sütunlar için "category" veri tipini kullanarak belleği optimize edelim
for col in birlesik_df.select_dtypes(include=['object']).columns:
    num_unique_values = len(birlesik_df[col].unique())
    num_total_values = len(birlesik_df[col])
    if num_unique_values / num_total_values < 0.5:
        birlesik_df[col] = birlesik_df[col].astype('category')


# Yeni ürünlerin sektör verisini yükle
new_sectors_df = pd.read_excel('1-) Yeni ürünlerin sektörleri.xlsx')

# Ülkeler verisini yükle
ulkeler_df = pd.read_excel('0-) Ülkeler.xlsx')  

# 2. Ürünleri Birleştir (6 basamaklı HS koduna göre birleştir)
merged_df = pd.merge(birlesik_df, new_sectors_df, how='left', left_on='k', right_on='6 Basamaklı HS Kodu')

# 3. Filtrele ve Ticaret Hacmini Hesapla
filtered_countries = ulkeler_df['country_code'].unique()  # Ülke kodlarını filtrele

# Yıl bazında ticaret hacmini hesapla
years = merged_df['t'].unique()
final_dfs = []

for year in years:
    # Yıla göre filtrele
    df_year = merged_df[merged_df['t'] == year]
    
    # İhracatçı ve ithalatçı ülkeleri filtrele
    df_filtered_year = df_year[(df_year['i'].isin(filtered_countries)) & (df_year['j'].isin(filtered_countries))]
    
    # 1 basamaklı HS sektörlerine göre ticaret hacmini hesapla
    yearly_totals = df_filtered_year.groupby(['t', 'i', '1 Basamaklı HS İsmi'])['v'].sum().reset_index(name='sector_total')
    
    # Ülke bazında toplam ticaret hacmi hesapla
    yearly_totals['year_total'] = yearly_totals.groupby('i')['sector_total'].transform('sum')
    
    # Sektörün toplam ticaret hacmindeki oranını hesapla
    yearly_totals['k_1_oran'] = yearly_totals['sector_total'] / yearly_totals['year_total']
    
    final_dfs.append(yearly_totals)

# 4. Sonuçları Birleştir
final_df = pd.concat(final_dfs, ignore_index=True)

# Ülke kodlarını ülke adlarına dönüştür
final_df = final_df.merge(ulkeler_df[['country_code', 'Countries']], left_on='i', right_on='country_code', how='left')

# Ülke kodu yerine ülke adlarını ekle
final_df['Ülke'] = final_df['Countries']

# 5. RCA Analizi Yap
# Dünya toplam ticaret hacmi (tüm ülkeler ve sektörler için)
world_totals = final_df.groupby(['Yıl', '1 Basamaklı HS İsimleri'])['Sektör Ticaret Hacmi'].sum().reset_index(name='world_sector_total')

# Dünya toplam ticaret hacmi (genel)
world_total_trade = final_df.groupby(['Yıl'])['Sektör Ticaret Hacmi'].sum().reset_index(name='world_total')

# Sektör bazında RCA hesaplama
final_df = final_df.merge(world_totals, on=['Yıl', '1 Basamaklı HS İsimleri'], how='left')
final_df = final_df.merge(world_total_trade, on=['Yıl'], how='left')

# RCA Formülü Uygulaması
final_df['RCA'] = (final_df['Sektör Ticaret Hacmi'] / final_df['Genel Ticaret Hacmi']) / (final_df['world_sector_total'] / final_df['world_total'])

# Gereksiz sütunları çıkar ve sütun adlarını düzenle
final_df = final_df[['Yıl', 'Ülke', '1 Basamaklı HS İsimleri', 'Sektör Ticaret Hacmi', 'Genel Ticaret Hacmi', 'RCA']]

# 6. Excel Dosyasına Kaydet
output_file_path = "aababa.xlsx"
final_df.to_excel(output_file_path, index=False)

print(f"Sonuçlar '{output_file_path}' dosyasına kaydedildi.")


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000001E89DA5AB10>>
Traceback (most recent call last):
  File "C:\Users\oguzo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel\ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


MemoryError: Unable to allocate 803. MiB for an array with shape (1, 105197091) and data type float64

In [11]:
import pandas as pd

# Yeni ürünlerin sektör verisini yükle
new_sectors_df = pd.read_excel('1-) Yeni ürünlerin sektörleri.xlsx')

# Ülkeler verisini yükle
ulkeler_df = pd.read_excel('0-) Ülkeler.xlsx')  

# 2. Ürünleri Birleştir (6 basamaklı HS koduna göre birleştir)
merged_df = pd.merge(birlesik_df, new_sectors_df, how='left', left_on='k', right_on='6 Basamaklı HS Kodu')

# 3. Filtrele ve Ticaret Hacmini Hesapla
filtered_countries = ulkeler_df['country_code'].unique()  # Ülke kodlarını filtrele

# Yıl bazında ticaret hacmini hesapla
years = merged_df['t'].unique()
final_dfs = []

for year in years:
    # Yıla göre filtrele
    df_year = merged_df[merged_df['t'] == year]
    
    # İhracatçı ve ithalatçı ülkeleri filtrele
    df_filtered_year = df_year[(df_year['i'].isin(filtered_countries)) & (df_year['j'].isin(filtered_countries))]
    
    # 1 basamaklı HS sektörlerine göre ticaret hacmini hesapla
    yearly_totals = df_filtered_year.groupby(['t', 'i', '1 Basamaklı HS İsmi'])['v'].sum().reset_index(name='sector_total')
    
    # Ülke bazında toplam ticaret hacmi hesapla
    yearly_totals['year_total'] = yearly_totals.groupby('i')['sector_total'].transform('sum')
    
    # Sektörün toplam ticaret hacmindeki oranını hesapla
    yearly_totals['k_1_oran'] = yearly_totals['sector_total'] / yearly_totals['year_total']
    
    final_dfs.append(yearly_totals)

# 4. Sonuçları Birleştir
final_df = pd.concat(final_dfs, ignore_index=True)

# Ülke kodlarını ülke adlarına dönüştür
final_df = final_df.merge(ulkeler_df[['country_code', 'Countries']], left_on='i', right_on='country_code', how='left')

# Ülke kodu yerine ülke adlarını ekle
final_df['Ülke'] = final_df['Countries']

# 5. RCA Analizi Yap
# Dünya toplam ticaret hacmi (tüm ülkeler ve sektörler için)
world_totals = final_df.groupby(['Yıl', '1 Basamaklı HS İsimleri'])['Sektör Ticaret Hacmi'].sum().reset_index(name='world_sector_total')

# Dünya toplam ticaret hacmi (genel)
world_total_trade = final_df.groupby(['Yıl'])['Sektör Ticaret Hacmi'].sum().reset_index(name='world_total')

# RCA hesaplama: Dünya ve ülke ticaret hacmi oranlarına göre
final_df = pd.concat([final_df, world_totals, world_total_trade], axis=1)

# RCA Formülü Uygulaması
final_df['RCA'] = (final_df['Sektör Ticaret Hacmi'] / final_df['Genel Ticaret Hacmi']) / (final_df['world_sector_total'] / final_df['world_total'])

# Gereksiz sütunları çıkar ve sütun adlarını düzenle
final_df = final_df[['Yıl', 'Ülke', '1 Basamaklı HS İsimleri', 'Sektör Ticaret Hacmi', 'Genel Ticaret Hacmi', 'RCA']]

# 6. Excel Dosyasına Kaydet
output_file_path = "aababa.xlsx"
final_df.to_excel(output_file_path, index=False)

print(f"Sonuçlar '{output_file_path}' dosyasına kaydedildi.")


MemoryError: Unable to allocate 3.92 GiB for an array with shape (5, 105197091) and data type object